In [1]:
import re
import pandas as pd
import enchant

d = enchant.Dict("en_US")

In [3]:
raw_ner_result = pd.read_csv('./NER/304.txt_test_knowledge_base.csv', header=None, encoding='utf-8')

In [4]:
raw_ner_result

,0,1
0,"techniques , in conjunction with X - ray","A variety of electrochemical techniques , in c..."
1,photoelectron,"A variety of electrochemical techniques , in c..."
2,photoelectron,The open - top cells were analyzed using X - r...
3,"spectroscopy , support these conclusions .","A variety of electrochemical techniques , in c..."
4,impedance,All symmetric cells were conditioned by cyclin...
...,...,...
268,electrochemical,"At the same time , the Cr layers promoted reve..."
269,conversion of the,"At the same time , the Cr layers promoted reve..."
270,stability and interphase,Experiments with electron - blocking Cr - AlO ...
271,growth were similar Cr alone .,Experiments with electron - blocking Cr - AlO ...


In [5]:
filter_list = []
for i in range(len(raw_ner_result)):
    now_word_list = str(raw_ner_result.iloc[i, 0]).split(" ")
    
    count = 0
    for j in range(len(now_word_list)):
        if d.check(now_word_list[j]):
            count += 1
                
    if count == len(now_word_list):
        filter_list.append([raw_ner_result.iloc[i, 0], raw_ner_result.iloc[i, 1]])

In [6]:
len(filter_list)    

135

In [7]:
df_list = []
for ele in filter_list:
    if ele[0] in df_list:
        continue
    else:
        df_list.append(ele[0])

In [8]:
df = pd.DataFrame(df_list)
df.to_csv("./material_knowledge.csv", header=None, index=None, encoding='utf-8')

In [9]:
filter_list_df = pd.DataFrame(filter_list)

In [10]:
filter_list_df

,0,1
0,photoelectron,"A variety of electrochemical techniques , in c..."
1,photoelectron,The open - top cells were analyzed using X - r...
2,impedance,All symmetric cells were conditioned by cyclin...
3,spectra were collected in the range from MHz t...,All symmetric cells were conditioned by cyclin...
4,electron,Symmetric and full cells were studied ex situ ...
...,...,...
130,growth and delaying fracture at moderate current,Cr interlayer films were found to significantl...
131,electrochemical,"At the same time , the Cr layers promoted reve..."
132,conversion of the,"At the same time , the Cr layers promoted reve..."
133,stability and interphase,Experiments with electron - blocking Cr - AlO ...


In [11]:
merge_entity = {}

In [12]:
for i in range(len(filter_list_df)):
    now_sent = str(filter_list_df.iloc[i, 1]).split(" ")
    old_len = len(now_sent)
    last_word = now_sent.pop()
    new_len = len(now_sent)
    assert old_len-1 == new_len
    
    
    if last_word != "." or "." in now_sent or new_len <= 8:
        continue
    
    if filter_list_df.iloc[i, 1] not in merge_entity:
        merge_entity[filter_list_df.iloc[i, 1]] = [filter_list_df.iloc[i, 0]]
    else:
        merge_entity[filter_list_df.iloc[i, 1]].append(filter_list_df.iloc[i, 0])

In [13]:
len(merge_entity)

56

In [14]:
merge_entity

{'A variety of electrochemical techniques , in conjunction with X - ray photoelectron spectroscopy , support these conclusions .': ['photoelectron'],
 'The open - top cells were analyzed using X - ray photoelectron spectroscopy to determine the presence of lithium on the surface of the pellets .': ['photoelectron',
  'spectroscopy to determine the presence of lithium on the surface of the pellets .'],
 'Symmetric and full cells were studied ex situ using scanning electron microscopy .': ['electron',
  'microscopy .'],
 'The instrument used was a Zeiss Ultra FE - SEM with an accelerating voltage of or kV .': ['Ultra',
  'FE',
  'SEM with an accelerating',
  'voltage of or kV .'],
 'The surface of the sample was ﬂooded with slow + ions using the ﬂood gun to compensate for surface electrons and Ar charging .': ['gun to compensate for surface electrons and Ar charging .'],
 'This indicates the existence of a current density limit above which the Cr layer no longer enables a uniform and pla

In [15]:
pre_sent = []

In [16]:
for idx, (sent, entity_list) in enumerate(merge_entity.items()):
    entities = list(set(entity_list))

    if len(entities) < 2:
        continue
    else:
        temp = []
        for i in range(len(entities)-1):
            for j in range(i+1, len(entities)):
                temp.append((entities[i], entities[j]))
        for t in range(len(temp)):
            try:
                h0, h1 = sent.index(temp[t][0]), sent.index(temp[t][-1])
                t0, t1 = h0 + len(temp[t][0]) + 1, h1 + len(temp[t][-1]) + 1
                if h0 > h1:
                    e1_sent = sent[0:h0] + "<e2>" + temp[t][0] + "</e2> " + sent[t0:]
                    e2_sent = e1_sent[0:h1] + "<e1>" + temp[t][-1] + "</e1> " + e1_sent[t1:]
                    pre_sent.append(e2_sent)
                else:
                    e2_sent = sent[0:h1] + "<e2>" + temp[t][-1] + "</e2> " + sent[t1:]
                    e1_sent = e2_sent[0:h0] + "<e1>" + temp[t][0] + "</e1> " + e2_sent[t0:]
                    pre_sent.append(e1_sent)
            except:
                continue           

In [17]:
for ele in pre_sent:
    if "<e1>" not in ele:
        pre_sent.remove(ele)
        continue
    elif "</e1>" not in ele:
        pre_sent.remove(ele)
        continue
    elif "<e2>" not in ele:
        pre_sent.remove(ele)
        continue
    elif "</e2>" not in ele:
        pre_sent.remove(ele)
        continue

In [18]:
len(pre_sent)

104

In [20]:
for i in range(len(pre_sent)):
    with open("./test.tsv", 'a', encoding='utf-8') as f:
        f.write("Other")
        f.write("\t")
        f.write(pre_sent[i] + "\n")